![Hand with calculator](calculator.jpg)

Did you know that the average return from investing in stocks is [10% per year](https://www.nerdwallet.com/article/investing/average-stock-market-return) (not accounting for inflation)? But who wants to be average?! 

You have been asked to support an investment firm by analyzing trends in high-growth companies. They are interested in understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. Providing them with this information gives them a competitive insight as to industry trends and how they should structure their portfolio looking forward.

You have been given access to their `unicorns` database, which contains the following tables:

## dates
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `date_joined` | The date that the company became a unicorn.  |
| `year_founded` | The year that the company was founded.       |

## funding
| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| `company_id`       | A unique ID for the company.                 |
| `valuation`        | Company value in US dollars.                 |
| `funding`          | The amount of funding raised in US dollars.  |
| `select_investors` | A list of key investors in the company.      |

## industries
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `industry`     | The industry that the company operates in.   |

## companies
| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| `company_id`   | A unique ID for the company.                      |
| `company`      | The name of the company.                          |
| `city`         | The city where the company is headquartered.      |
| `country`      | The country where the company is headquartered.   |
| `continent`    | The continent where the company is headquartered. |


# The output

Your query should return a table in the following format:
| industry  | year | num\_unicorns       | average\_valuation\_billions |
| --------- | ---- | ------------------- | ---------------------------- |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |

Where `industry1`, `industry2`, and `industry3` are the three top-performing industries.

In [3]:
select * from public.industries


,company_id,industry
0,189,Health
1,848,Fintech
2,556,Internet software & services
3,999,Internet software & services
4,396,Fintech
...,...,...
1069,1002,Mobile & telecommunications
1070,215,E-commerce & direct-to-consumer
1071,866,Internet software & services
1072,651,Fintech


In [4]:
WITH top_industries AS (
    -- Step 1: Identify the top three industries based on the number of new unicorns in 2019, 2020, 2021
    SELECT
        industry,
        COUNT(*) AS total_unicorns
    FROM
        public.funding AS pf
    INNER JOIN
        public.industries AS pi USING (company_id)
    INNER JOIN
        public.companies AS pc USING (company_id)
    INNER JOIN
        public.dates AS pd USING (company_id)
    WHERE
        pf.valuation >= 1000000000
        AND date_part('year', date_joined) IN (2019, 2020, 2021)
    GROUP BY
        industry
    ORDER BY
        total_unicorns DESC
    LIMIT 3
	),
	
cte as (
select industry,date_part('year',date_joined) as year,count(*) as num_unicorns,round(avg(valuation)/1000000000,2) as average_valuation_billions 
from public.funding as pf 
inner join public.industries as pi
using(company_id)
inner join public.companies as pc
using(company_id)
inner join public.dates pd
using(company_id)
where pf.valuation >= 1000000000 and date_part('year',date_joined) in (2019,2020,2021) and industry in (select industry from top_industries)
group by industry,year

)
select *  from cte
order by year desc,num_unicorns desc





,industry,year,num_unicorns,average_valuation_billions
0,Fintech,2021,138,2.75
1,Internet software & services,2021,119,2.15
2,E-commerce & direct-to-consumer,2021,47,2.47
3,Internet software & services,2020,20,4.35
4,E-commerce & direct-to-consumer,2020,16,4.00
5,Fintech,2020,15,4.33
6,Fintech,2019,20,6.80
7,Internet software & services,2019,13,4.23
8,E-commerce & direct-to-consumer,2019,12,2.58
